In [ ]:
# Khai báo các thư viện cần thiết

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pickle
import numpy as np

In [ ]:
# Mạng Nơ ron 

class MazeNetCombined(nn.Module):
    def __init__(self, local_size=11, global_size=10, num_actions=4):
        super(MazeNetCombined, self).__init__()
        
        # Quan sát cục bộ
        self.conv1_local = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2_local = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3_local = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4_local = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Quan sát toàn cục
        self.conv1_global = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2_global = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3_global = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4_global = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Fully Connected cho vị trí hiện tại
        self.fc_position = nn.Linear(2, 32)

        # Tầng Fully Connected cuối cùng
        self.fc1 = nn.Linear(256 * local_size * local_size + 256 * global_size * global_size + 32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout_fc = nn.Dropout(p=0.5)  # Dropout trước tầng FC3
        self.fc3 = nn.Linear(128, num_actions)

    def forward(self, local_obs, global_obs, position):
        # Xử lý local_obs
        x_local = F.relu(self.conv1_local(local_obs))
        x_local = F.relu(self.conv2_local(x_local))
        x_local = F.relu(self.conv3_local(x_local))
        x_local = F.relu(self.conv4_local(x_local))
        x_local = x_local.view(x_local.size(0), -1)
    
        # Xử lý global_obs
        x_global = F.relu(self.conv1_global(global_obs))
        x_global = F.relu(self.conv2_global(x_global))
        x_global = F.relu(self.conv3_global(x_global))
        x_global = F.relu(self.conv4_global(x_global))
        x_global = x_global.view(x_global.size(0), -1)
        # Xử lý vị trí hiện tại
        x_position = F.relu(self.fc_position(position))

        # Kết hợp tất cả
        x = torch.cat((x_local, x_global, x_position), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout_fc(x)  # Dropout trước FC3
        x = self.fc3(x)

        return x


In [ ]:
# Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []

    def push(self, experience):
        """Thêm một trải nghiệm (local_obs, global_obs, position, action, reward, next_local_obs, next_global_obs, next_position, done) vào replay buffer."""
        if len(self.buffer) >= self.capacity:
            self.buffer.pop(0)  # Loại bỏ trải nghiệm cũ nhất nếu đầy
        self.buffer.append(experience)

    def sample(self, batch_size):
        """Trích xuất batch mẫu từ replay buffer."""
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[idx] for idx in indices]
        # Tách dữ liệu thành các phần riêng biệt
        local_obs, global_obs, position, actions, rewards, next_local_obs, next_global_obs, next_position, dones = zip(*batch)
        return (np.array(local_obs), np.array(global_obs), np.array(position), 
                np.array(actions), np.array(rewards), np.array(next_local_obs), 
                np.array(next_global_obs), np.array(next_position), np.array(dones))

    def __len__(self):
        return len(self.buffer)

In [ ]:
# Cập nhật model
def update_model(policy_net, target_net, replay_buffer, optimizer, batch_size, gamma, device):
    """
    Cập nhật mô hình chính (policy network) cho mạng có đầu vào đa dạng (local_obs, global_obs, position).

    Args:
    - policy_net (nn.Module): Mạng chính dự đoán giá trị Q(s, a).
    - target_net (nn.Module): Mạng mục tiêu dùng để tính Q_target.
    - replay_buffer (ReplayBuffer): Bộ nhớ replay chứa các kinh nghiệm (local_obs, global_obs, position, action, reward, next_local_obs, next_global_obs, next_position, done).
    - optimizer (torch.optim.Optimizer): Trình tối ưu hóa (Adam, SGD, ...).
    - batch_size (int): Kích thước batch mẫu từ replay buffer.
    - gamma (float): Hệ số chiết khấu (discount factor).
    - device (torch.device): Thiết bị thực thi (CPU hoặc GPU).

    Returns:
    - loss (float): Giá trị mất mát (loss) sau khi cập nhật.
    """
    # Kiểm tra nếu replay buffer chưa đủ dữ liệu
    if len(replay_buffer) < batch_size:
        return None

    # 1. Lấy mẫu từ replay buffer
    batch = replay_buffer.sample(batch_size)
    (local_obs, global_obs, position, actions, rewards, 
     next_local_obs, next_global_obs, next_position, dones) = batch

    # 2. Chuyển đổi dữ liệu sang Tensor và đưa vào thiết bị (CPU/GPU)
    local_obs = torch.tensor(local_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh cho CNN
    global_obs = torch.tensor(global_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    position = torch.tensor(position, dtype=torch.float32).to(device)
    actions = torch.tensor(actions, dtype=torch.long).to(device)
    rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
    next_local_obs = torch.tensor(next_local_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    next_global_obs = torch.tensor(next_global_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    next_position = torch.tensor(next_position, dtype=torch.float32).to(device)
    dones = torch.tensor(dones, dtype=torch.float32).to(device)

    # 3. Dự đoán Q(s, a) từ policy_net
    q_values = policy_net(local_obs, global_obs, position)  # Đầu ra: (batch_size, num_actions)
    q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)  # Lấy giá trị Q(s, a) cho hành động đã thực hiện

    # 4. Tính toán Q_target bằng target_net
    with torch.no_grad():
        next_q_values = target_net(next_local_obs, next_global_obs, next_position)  # Dự đoán Q(s', a') từ target_net
        max_next_q_values = next_q_values.max(1)[0]  # Lấy giá trị lớn nhất Q(s', a')
        q_targets = rewards + gamma * max_next_q_values * (1 - dones)  # Hàm Bellman

    # 5. Tính hàm mất mát
    loss = F.mse_loss(q_values, q_targets)

    # 6. Tối ưu hóa mô hình
    optimizer.zero_grad()  # Xóa gradient cũ
    loss.backward()  # Lan truyền ngược (backpropagation)
    optimizer.step()  # Cập nhật trọng số

    return loss.item()


In [ ]:
# Đồng bộ với mạng mục tiêu
def sync_target_network(q_network, target_network):
    target_network.load_state_dict(q_network.state_dict())

In [ ]:
# Khởi tạo các siêu tham số
import torch.optim as optim

# Thiết bị thực thi (CPU hoặc GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

# Hyperparameters
gamma = 0.8  # Hệ số chiết khấu (discount factor)
learning_rate = 1e-3  # Learning rate cho optimizer
weight_decay = 1e-3 # weight decay cho optimizer
batch_size = 64  # Kích thước batch khi lấy mẫu từ replay buffer
num_epochs = 10000             # Số epoch huấn luyện
replay_buffer_capacity = 5000  # Dung lượng bộ nhớ replay buffer

# Mạng chính và mạng mục tiêu
local_size = 11  # Kích thước quan sát cục bộ
global_size = 8  # Kích thước quan sát toàn cục (downsample)
num_actions = 4  # Số hành động (lên, xuống, trái, phải)

# Khởi tạo mạng chính (policy_net) và mạng mục tiêu (target_net)
policy_net = MazeNetCombined(local_size=local_size, global_size=global_size, num_actions=num_actions).to(device)
target_net = MazeNetCombined(local_size=local_size, global_size=global_size, num_actions=num_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())  # Đồng bộ hóa trọng số ban đầu
target_net.eval()  # Đặt target_net ở chế độ đánh giá

# Optimizer
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate, weight_decay = weight_decay)

print("Initialization complete!")

In [ ]:
# Khởi tạo replay buffer

with open("replay_buffer.pkl", "rb") as file:
    replay_buffer = pickle.load(file)

print(f"Replay buffer loaded with {len(replay_buffer)} experiences.")

In [ ]:
# Lưu, tải trọng số của mô hình

# Lưu trọng số của mô hình
def save_state_dict(model, path="model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model weight was saved to: {path}")

# Tải trọng số của mô hình
def load_state_dict(model, target_model, path = "model.pth"):
    model.load_state_dict(torch.load(path))
    target_model.load_state_dict(model.state_dict())  # Đồng bộ hóa trọng số ban đầu
    target_model.eval() # Đặt target_model ở chế độ đánh giá
    print(F"Model weight was loaded from: {path}")

load_state_dict(policy_net, target_net)

In [ ]:
# Huấn luyện mô hình trên replay buffer
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    loss = update_model(policy_net, target_net, replay_buffer, optimizer, batch_size, gamma, device)
    
    if loss is not None:
        print(f"Loss: {loss:.4f}")
    else:
        print("Replay buffer không đủ dữ liệu để cập nhật mô hình.")

    # Đồng bộ hóa mạng mục tiêu
    if epoch % 100 == 0:
        sync_target_network(policy_net, target_net)

# Lưu trọng số của mô hình
save_state_dict(policy_net, path=f"model.pth")

In [ ]:
# Xuất mô hình sang định dạng ONNX
import onnx
import onnxruntime

def export_to_onnx(trained_model, device, local_size=11, global_size=10, num_actions=4, output_file="model.onnx"):
    """
    Xuất mô hình PyTorch đã huấn luyện sang định dạng ONNX.

    Args:
    - trained_model (nn.Module): Mô hình PyTorch đã huấn luyện.
    - local_size (int): Kích thước quan sát cục bộ (ví dụ: 11x11).
    - global_size (int): Kích thước quan sát toàn cục (ví dụ: 10x10).
    - num_actions (int): Số lượng hành động trong mê cung.
    - output_file (str): Đường dẫn tệp để lưu mô hình ONNX.

    Returns:
    - None: Mô hình ONNX được lưu vào file.
    """
    # Đặt mô hình ở chế độ đánh giá
    trained_model.to(device)
    trained_model.eval()

    # Tạo dữ liệu đầu vào giả (dummy input) để định hình đầu vào
    dummy_local_obs = torch.randn(1, 1, local_size, local_size).to(device)  # Local observation
    dummy_global_obs = torch.randn(1, 1, global_size, global_size).to(device)  # Global observation
    dummy_position = torch.randn(1, 2).to(device)  # Vị trí hiện tại
    
    # Xuất mô hình sang định dạng ONNX
    torch.onnx.export(
        trained_model,  # Mô hình đã huấn luyện
        (dummy_local_obs, dummy_global_obs, dummy_position),  # Đầu vào
        output_file,  # Tên tệp ONNX
        input_names=["local_obs", "global_obs", "position"],  # Tên các đầu vào
        output_names=["action"],  # Tên đầu ra
        dynamic_axes={
            "local_obs": {0: "batch_size"}, 
            "global_obs": {0: "batch_size"}, 
            "position": {0: "batch_size"}, 
            "action": {0: "batch_size"}
        }
    )
    print(f"Model exported to ONNX format: {output_file}")

# Xuất mô hình sang ONNX
export_to_onnx(target_net, device, local_size, global_size, num_actions, "maze_net_combined.onnx")